<a href="https://colab.research.google.com/github/VitorSancho/Projeto_Eleicoes2020RJ/blob/main/Estudo_Elei%C3%A7%C3%B5es_2020i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eleições 2020 - Análise do município de Rio de Janeiro 

## Imports

In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Funções


# A função abaixo foi copiada dos códigos do Luis Moneda: https://www.kaggle.com/lgmoneda/reported-assets-by-party

# Formatando o Valor dos bens para o modelo R$ ####,##
def human_format(num, pos):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return 'R${}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [25]:
from google.colab import drive
# drive.mount('/content/drive/My Drive/consulta_cand_2020_RJ_def.csv')

# Leitura de arquivos e criação de séries booleanas

In [27]:
candidatos_rj = pd.read_csv("/content/drive/My Drive/consulta_cand_2020_RJ_def.csv",sep=';', encoding='iso-8859-1')
bens_candidatos_rj = pd.read_csv("/content/drive/My Drive/bem_candidato_2020_BRASIL.csv", sep=';', encoding='iso-8859-1')

# Criando série booleanas para focar análise no município do Rio de Janeiro
selecao_municipio=candidatos_rj['NM_UE']=="RIO DE JANEIRO"
selecao_municipio_bens=bens_candidatos_rj['NM_UE']=="RIO DE JANEIRO"

# Filtrando arquivo de bens com dados do município do Rio de Janeiro
bens_dos_candidatos_municipio_rj=bens_candidatos_rj[selecao_municipio_bens]

# Salvando o arquivo para trabalhos futuros
#bens_dos_candidatos_municipio_rj.to_csv('bens_dos_candidatos_municipio_rj.csv',sep=';',index=False)


In [ ]:
bens_dos_candidatos_municipio_rj.head()
#candidatos_rj.head()
#bens_candidatos_rj.head()
#candidatos_rj.head()

# N° de candidatos dos 20 maiores partidos do município do RJ

In [ ]:
candidatos_municipio_rj=candidatos_rj[selecao_municipio]
candidatos_municipio_rj['SG_PARTIDO'].value_counts().head(20).plot.bar(figsize=(10,5))
plt.xticks(rotation=60)
plt.ylim((50,85))
plt.ylabel("N° de candidatos", size = 16)

plt.title("Os 20 partidos com maior n° de candidatos no município do RJ", 
          fontdict={'family': 'serif', 
                    'weight': 'bold',
                    'size': 16});

# Candidato de maior idade e respectivo cargo a qual concorre

In [ ]:
idade=candidatos_municipio_rj["NR_IDADE_DATA_POSSE"].max()
selecao_candidade_de_maior_idade=candidatos_rj['NR_IDADE_DATA_POSSE']==idade
cargo=candidatos_rj[selecao_candidade_de_maior_idade]['DS_CARGO']


Vemos que na verdade são dois candidatos de 86 anos, um concorrendo como vereador e outro como vice-prefeito.

# Distribuição de bens declarados por candidatos por partido

In [ ]:
# Para essa análise baseei-me nas análises realizadas pelo Luis Moneda 
# Não possuo numa mesma tabela as colunas VG_BEM_CANDIDATO e SG_PARTIDO
# Para resolver esse problema devo realizar um join entre os data frames candidatos_rj e bens_dos_candidatos_municipio_rj

# Pegando somente as colunas que são de interesse de cada dataframe 
candidatos_rj_dados_fatiados = candidatos_rj[["SQ_CANDIDATO", "NM_URNA_CANDIDATO", "SG_PARTIDO"]]
bens_candidatos_dados_fatiados=bens_dos_candidatos_municipio_rj[["SQ_CANDIDATO",'VR_BEM_CANDIDATO']]

# Realizando o join utilizando como chave a coluna SQ_CANDIDATO
concatenado_partido_bem_candidato = bens_candidatos_dados_fatiados.merge(candidatos_rj_dados_fatiados, on="SQ_CANDIDATO")

# Os dados da coluna VR_BEM_CANDIDATO estão no formato object. 
# Como esse dado será utilizado na construção do boxplot, transformo-o em tipo float  
concatenado_partido_bem_candidato["VR_BEM_CANDIDATO"] = concatenado_partido_bem_candidato["VR_BEM_CANDIDATO"].apply(lambda x: x.split(",")[0]).astype(float)

#concatenado_partido_bem_candidato.info()



In [ ]:
# Tive problemas em criar uma tebela com o total de bens por candidato por partido
# Por isso, tive que fatiar o dataframe obtido anteriormente
# Utilizando o dataframe fatiado e o dataframe candidatos_rj adicionei novamente o dado SG_PARTIDO ao dataframe que utilizarei para construir o boxplot

# Peguei todos os dados do ultimo dataframe obtido, excluindo a coluna de sigla do partido
concatenado_bem_candidato=concatenado_partido_bem_candidato[['VR_BEM_CANDIDATO',"NM_URNA_CANDIDATO","SG_PARTIDO"]]
agrupado_valor_de_bens_por_candidato=concatenado_bem_candidato.groupby(['NM_URNA_CANDIDATO']).sum()

concatenado_partido_bem_candidato_new = agrupado_valor_de_bens_por_candidato.merge(candidatos_rj_dados_fatiados, on="NM_URNA_CANDIDATO")

#concatenado_partido_bem_candidato_new

In [ ]:
# tratando outliers


grupo_partido=concatenado_partido_bem_candidato_new.groupby('SG_PARTIDO')['VR_BEM_CANDIDATO']

#calculando os quartis
Q1=grupo_partido.quantile(.25) #Pegando o valor da posição 1/4 da distribuição de valor
Q3=grupo_partido.quantile(.75) #Pegando o valor da posição 3/4 da distribuição de valor

#intervalo entre os quartis
IIQ=Q3-Q1
limite_inferior=Q1-1.5*IIQ
limite_superior=Q3+1.5*IIQ

concatenado_new=pd.DataFrame()
for partido in grupo_partido.groups.keys():
    eh_tipo=concatenado_partido_bem_candidato_new['SG_PARTIDO']==partido #Criando uma série booleana
    eh_dentro_limite=(concatenado_partido_bem_candidato_new['VR_BEM_CANDIDATO']>=limite_inferior[partido])&(concatenado_partido_bem_candidato_new['VR_BEM_CANDIDATO']<=limite_superior[partido]) 
    #criando outra série booleana que leva em consideração o tratamento de outliers 
    selecao=eh_tipo & eh_dentro_limite #concatenando as duas series booleanas acima
    dados_selecao=concatenado_partido_bem_candidato_new[selecao]
    concatenado_new=pd.concat([concatenado_new,dados_selecao]) # adicionando tabela ao dataframe em branco criado antes do for



In [ ]:

fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(data=concatenado_new, y="VR_BEM_CANDIDATO", x="SG_PARTIDO",
            order=['DEM',"PSOL", "PDT", "MDB", "NOVO"],palette="Blues")


ax.yaxis.set_major_formatter(ticker.FuncFormatter(human_format))
plt.ylim([0, 1500000])
plt.title("Distribuição dos bens declarados por candidatos - Município do Rio de Janeiro", fontsize=15, pad=16)
plt.xlabel("Partido",size = 15)
plt.ylabel("Valor dos bens declarados",size = 15)

plt.show()

# Distribuição de grau de escolaridade por partido

# Situação da candidatura


In [ ]:
selecao_situacao=candidatos_rj['DS_SITUACAO_CANDIDATURA']=="INAPTO"
situacao=candidatos_municipio_rj[selecao_situacao]

quantidade_total_de_candidatos=candidatos_municipio_rj.shape[0]
quantidade_de_candidatos_inaptos=situacao.shape[0]

print("{} candidatos de um total de {} estão inaptos a concorrer.". format(quantidade_de_candidatos_inaptos,quantidade_total_de_candidatos))

Temos cerca de 3,7% de candidaturas inaptas no munício do Rio de Janeiro.

In [ ]:
grafico4=situacao['SG_PARTIDO'].value_counts().plot.bar(figsize=(10,5))
plt.xticks(rotation=80)
plt.ylabel("N° de candidatos inaptos", size = 16)

plt.title("N° de candidatos inaptos por partidos", 
          fontdict={'family': 'serif', 
                    'weight': 'bold',
                    'size': 16});

grafico4;                    

# Organizando Plots para publicação

In [ ]:
area=plt.figure()

g1=area.add_subplot(2,2,1)
g2=area.add_subplot(2,2,2)
g3=area.add_subplot(2,2,3)
g4=area.add_subplot(2,2,4)


g4=grafico4.show

# Referências e Links

In [ ]:
Pesquisas Eleitorais: https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais
Página Kaggle com datasets do Luis Moneda: https://www.kaggle.com/lgmoneda/brazil-elections-2020